In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
from model import build_unet
from metrics import dice_coef, iou

In [2]:
H = 256
W = 256

In [3]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

In [4]:
def load_data(dataset_path, split=(0.6, 0.2, 0.2)):
    images_path = os.path.join(dataset_path, "ISIC2018_Task1-2_Validation_Input")
    masks_path = os.path.join(dataset_path, "ISIC2018_Task1_Validation_GroundTruth")

    print("Images Path:", images_path)
    print("Masks Path:", masks_path)

    images = sorted(glob(os.path.join(images_path, "*.jpg")))
    masks = sorted(glob(os.path.join(masks_path, "*.png")))

    print("Number of Images:", len(images))
    print("Number of Masks:", len(masks))

    assert len(images) > 0, "No images found in the dataset path."

    train_split, valid_split, test_split = split
    train_size = int(len(images) * train_split)
    valid_size = int(len(images) * valid_split)

    train_x, valid_x, test_x = np.split(images, [train_size, train_size + valid_size])
    train_y, valid_y, test_y = np.split(masks, [train_size, train_size + valid_size])

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)


In [5]:
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)  ## (H, W, 3)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x    

In [6]:
def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (H, W)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)                    ## (256, 256)
    x = np.expand_dims(x, axis=-1)              ## (256, 256, 1)
    return x

In [7]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

In [8]:
def tf_dataset(X, Y, batch):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset

In [9]:

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Folder for saving data """
    create_dir("files")

    """ Hyperparameters """
    batch_size = 4
    lr = 1e-4 ## (0.0001)
    num_epoch = 5
    model_path = "files/model.h5"
    csv_path = "files/data.csv"

    """ Dataset : 60/20/20 """
    dataset_path = "/Users/shravanisajekar/Desktop/unet_dataset"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path, split=(0.6, 0.2, 0.2))


    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Test: {len(test_x)} - {len(test_y)}")

    train_dataset = tf_dataset(train_x, train_y, batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch_size)

    train_steps = len(train_x)//batch_size
    valid_steps = len(valid_x)//batch_size

    if len(train_x) % batch_size != 0:
        train_steps += 1

    if len(valid_x) % batch_size != 0:
        valid_steps += 1

    """ Model """
    model = build_unet((H, W, 3))
    metrics = [dice_coef, iou, Recall(), Precision()]
    model.compile(loss="binary_crossentropy", optimizer=Adam(lr), metrics=metrics)
    model.summary()

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path),
        TensorBoard(),
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
    ]

    model.fit(
        train_dataset,
        epochs=num_epoch,
        validation_data=valid_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks
    )

Images Path: /Users/shravanisajekar/Desktop/unet_dataset/ISIC2018_Task1-2_Validation_Input
Masks Path: /Users/shravanisajekar/Desktop/unet_dataset/ISIC2018_Task1_Validation_GroundTruth
Number of Images: 100
Number of Masks: 100
Train: 60 - 60
Test: 20 - 20


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 256, 256, 64)         1792      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 256, 256, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 256, 256, 64)         0         ['batch_normalization[0][0

 chNormalization)                                                                                 
                                                                                                  
 activation_8 (Activation)   (None, 16, 16, 1024)         0         ['batch_normalization_8[0][0]'
                                                                    ]                             
                                                                                                  
 conv2d_9 (Conv2D)           (None, 16, 16, 1024)         9438208   ['activation_8[0][0]']        
                                                                                                  
 batch_normalization_9 (Bat  (None, 16, 16, 1024)         4096      ['conv2d_9[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 activatio

 )                                                                   'activation_1[0][0]']        
                                                                                                  
 conv2d_16 (Conv2D)          (None, 256, 256, 64)         73792     ['concatenate_3[0][0]']       
                                                                                                  
 batch_normalization_16 (Ba  (None, 256, 256, 64)         256       ['conv2d_16[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_16 (Activation)  (None, 256, 256, 64)         0         ['batch_normalization_16[0][0]
                                                                    ']                            
                                                                                                  
 conv2d_17

/Users/shravanisajekar/miniconda3/envs/codehub/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


15/15 [==============================] - 14s 817ms/step - loss: 0.5222 - dice_coef: 0.5216 - iou: 0.3654 - recall: 0.7619 - precision: 0.5619 - val_loss: 0.6887 - val_dice_coef: 0.3170 - val_iou: 0.1906 - val_recall: 0.3461 - val_precision: 0.4071 - lr: 1.0000e-04
Epoch 2/5
15/15 [==============================] - ETA: 0s - loss: 0.3821 - dice_coef: 0.6194 - iou: 0.4643 - recall: 0.7767 - precision: 0.7684
Epoch 2: val_loss improved from 0.68869 to 0.67823, saving model to files/model.h5
15/15 [==============================] - 12s 805ms/step - loss: 0.3821 - dice_coef: 0.6194 - iou: 0.4643 - recall: 0.7767 - precision: 0.7684 - val_loss: 0.6782 - val_dice_coef: 0.3113 - val_iou: 0.1865 - val_recall: 0.0017 - val_precision: 0.0588 - lr: 1.0000e-04
Epoch 3/5
15/15 [==============================] - ETA: 0s - loss: 0.3340 - dice_coef: 0.6498 - iou: 0.4977 - recall: 0.8001 - precision: 0.8104
Epoch 3: val_loss improved from 0.67823 to 0.63217, saving model to files/model.h5
15/15 [=======

In [11]:
# Save the trained model including architecture and weights
model.save("unet_model.h5")
